In [134]:
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
import keras
print("Tensorflow version: " + tf.__version__)
print("Keras version: " + keras.__version__)

Tensorflow version: 2.1.3
Keras version: 2.3.0


In [2]:
# Load data
tic = time.time()

# For local machine:
# path = r'C:\Users\Nick Bashour\Documents\Personal\14. Stanford\2. Academics\3. 2021 Spring\1. CS 230\2. Project\3. Code\\'
# For AWS EC2 instance:
path = "clean_data/"

X_train = np.genfromtxt(str(path+'X_train.csv'), delimiter=',')
Y_train = np.genfromtxt(str(path+'Y_train.csv'), delimiter=',')

X_test = np.genfromtxt(str(path+'X_test.csv'), delimiter=',')
Y_test = np.genfromtxt(str(path+'Y_test.csv'), delimiter=',')

toc = time.time()
print("time elapsed: " + str(toc-tic) + " sec's")

time elapsed: 353.20157623291016 sec's


In [3]:
# Data dimensions
print("Training set shapes: ")
print("   X: " + str(X_train.shape))
print("   Y: " + str(Y_train.shape))
print("Test set shapes: ")
print("   X: " + str(X_test.shape))
print("   Y: " + str(Y_test.shape))

nx = X_train.shape[1]
ny = Y_train.shape[1]
print("# of X features:",nx)
print("# of Y labels:", ny)

Training set shapes: 
   X: (2819023, 77)
   Y: (2819023, 15)
Test set shapes: 
   X: (11720, 77)
   Y: (11720, 15)
# of X features: 77
# of Y labels: 15


In [4]:
# Shuffle training data
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

tic = time.time()

X_train, Y_train = shuffle_in_unison(X_train, Y_train)

toc = time.time()

print("time elapsed: " + str(toc-tic) + " sec's")

time elapsed: 5.900663137435913 sec's


In [128]:
# Create labels
labels = np.array(['BENIGN', 'Bot', 'DDoS', 'DoS GoldenEye', 'DoS Hulk', 'DoS Slowhttptest',
 'DoS slowloris', 'FTP-Patator', 'Heartbleed', 'Infiltration', 'PortScan',
 'SSH-Patator', 'Web Attack - Brute Force', 'Web Attack - Sql Injection',
 'Web Attack - XSS'], dtype=object)

print("Unique label count: "+ str(len(labels)))

# Count # of samples by label
samples_of_label = [int(i) for i in np.sum(Y_train, axis = 0)]
print("# of samples by label:",samples_of_label)

Unique label count: 15
# of samples by label: [2272097, 966, 127027, 9293, 230073, 4499, 4796, 6938, 0, 0, 157930, 4897, 507, 0, 0]


In [129]:
# Penalties for each attack label are set at a baseline of the ratio of benign samples to that label's samples
# Additionally, hyperparameter to scale that baseline by a factor of scale_penalties
scale_penalties = 1000
class_penalties = np.zeros(len(labels))
for i in range(len(samples_of_label)):
    if samples_of_label[i] > 0:
        class_penalties[i] = samples_of_label[0] / samples_of_label[i]
    else:
        class_penalties[i] = 1
    class_penalties[0] = 1/scale_penalties

In [135]:
# The following blocks of code define the many models which were tested

In [130]:
def three_layer_model(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 30, activation = 'relu')(X_input)
    X = Dense(units = 20, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)    
    return model  

In [131]:
def four_layer_model(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 40, activation = 'relu')(X_input)    
    X = Dense(units = 30, activation = 'relu')(X)    
    X = Dense(units = 20, activation = 'relu')(X)        
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [132]:
def five_layer_model_1(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 30, activation = 'relu')(X_input)
    X = Dropout(rate = 0.2)(X)
    X = Dense(units = 24, activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
    X = Dense(units = 16, activation = 'relu')(X)    
    X = Dense(units = 10, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [133]:
def five_layer_model_2(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 50, activation = 'relu')(X_input)
    X = Dropout(rate = 0.2)(X)
    X = Dense(units = 40, activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
    X = Dense(units = 30, activation = 'relu')(X)    
    X = Dense(units = 20, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [57]:
def five_layer_model_3(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 50, activation = 'relu')(X_input)    
    X = Dense(units = 40, activation = 'relu')(X)    
    X = Dense(units = 30, activation = 'relu')(X)    
    X = Dense(units = 20, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [58]:
def six_layer_model_1(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 50, activation = 'relu')(X_input)    
    X = Dense(units = 40, activation = 'relu')(X)    
    X = Dense(units = 30, activation = 'relu')(X)    
    X = Dense(units = 25, activation = 'relu')(X)    
    X = Dense(units = 20, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [59]:
def six_layer_model_2(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 50, activation = 'relu')(X_input)    
    X = Dropout(rate=0.2)(X)
    X = Dense(units = 40, activation = 'relu')(X)    
    X = Dropout(rate=0.2)(X)
    X = Dense(units = 30, activation = 'relu')(X)    
    X = Dense(units = 25, activation = 'relu')(X)    
    X = Dense(units = 20, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [60]:
def six_layer_model_3(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 60, activation = 'relu')(X_input)    
    X = Dense(units = 50, activation = 'relu')(X)    
    X = Dense(units = 40, activation = 'relu')(X)    
    X = Dense(units = 30, activation = 'relu')(X)    
    X = Dense(units = 20, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [115]:
def six_layer_model_4(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 60, activation = 'relu')(X_input)    
    X = Dropout(rate = 0.2)(X)
    X = Dense(units = 50, activation = 'relu')(X)    
    X = Dropout(rate = 0.2)(X)
    X = Dense(units = 40, activation = 'relu')(X)    
    X = Dense(units = 30, activation = 'relu')(X)    
    X = Dense(units = 20, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [61]:
def seven_layer_model_1(input_shape):
    X_input = Input(shape = input_shape)
    X = Dense(units = 50, activation = 'relu')(X_input)
    X = Dense(units = 40, activation = 'relu')(X)    
    X = Dense(units = 30, activation = 'relu')(X)    
    X = Dense(units = 25, activation = 'relu')(X)
    X = Dense(units = 20, activation = 'relu')(X)        
    X = Dense(units = 15, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [62]:
def seven_layer_model_2(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 50, activation = 'relu')(X_input)
    X = Dropout(rate = 0.2)(X)
    X = Dense(units = 40, activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
    X = Dense(units = 30, activation = 'relu')(X)    
    X = Dense(units = 25, activation = 'relu')(X)    
    X = Dense(units = 20, activation = 'relu')(X)        
    X = Dense(units = 15, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [63]:
def seven_layer_model_3(input_shape):    
    X_input = Input(shape = input_shape)
    X = Dense(units = 70, activation = 'relu')(X_input)
    X = Dense(units = 60, activation = 'relu')(X)    
    X = Dense(units = 50, activation = 'relu')(X)    
    X = Dense(units = 40, activation = 'relu')(X)    
    X = Dense(units = 30, activation = 'relu')(X)        
    X = Dense(units = 20, activation = 'relu')(X)    
    X = Dense(units = len(labels), activation = 'softmax')(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [125]:
# Model set-up
model = six_layer_model_3(input_shape = (nx,))
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# First 25 epochs
tic = time.time()
model.fit(X_train, Y_train, batch_size = 128, epochs=25)
toc = time.time()
print("First 25 epochs elapsed: ", str(toc-tic), " seconds")

# Measure accuracy on test data
_, test_acc = model.evaluate(X_test, Y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# Save model for future use
path = 'saved_models/'
name = '210603_6_L_3_25e.h5'
model.save(path+name)

# Next 25 epochs
model.fit(X_train, Y_train, batch_size = 128, epochs=25)
toc = time.time()
print("50 epochs elapsed: ", str(toc-tic), " seconds")

# Measure accuracy on test data
_, test_acc = model.evaluate(X_test, Y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# Save model for future use
path = 'saved_models/'
name = '210603_6_L_3_50e.h5'
model.save(path+name)

Epoch 1/25
2819023/2819023 [==============================] - 108s 38us/step - loss: 0.1381 - accuracy: 0.9557
Epoch 2/25
2819023/2819023 [==============================] - 105s 37us/step - loss: 0.0703 - accuracy: 0.9716
Epoch 3/25
2819023/2819023 [==============================] - 106s 38us/step - loss: 0.0636 - accuracy: 0.9736
Epoch 4/25
2819023/2819023 [==============================] - 105s 37us/step - loss: 0.0592 - accuracy: 0.9756
Epoch 5/25
2819023/2819023 [==============================] - 106s 38us/step - loss: 0.0551 - accuracy: 0.9775
Epoch 6/25
2819023/2819023 [==============================] - 110s 39us/step - loss: 0.0496 - accuracy: 0.9808
Epoch 7/25
2819023/2819023 [==============================] - 110s 39us/step - loss: 0.0434 - accuracy: 0.9839
Epoch 8/25
2819023/2819023 [==============================] - 105s 37us/step - loss: 0.0401 - accuracy: 0.9848
Epoch 9/25
2819023/2819023 [==============================] - 107s 38us/step - loss: 0.0388 - accuracy: 0.9851
E